In [2]:
import pandas as pd
import os
import itertools
import xlsxwriter
from rapidfuzz import fuzz

os.chdir('M:\GY number duplicates')
pd.set_option('display.max_columns', 1000, 'display.width', 1000, 'display.max_rows',1000)
demogs = df = pd.read_csv('GY_update.txt', '\t')

In [3]:
# Only rows with GY numbers
demogs = demogs[demogs['GY_Number'].str.contains('GY')]

# Ensure no duplicates
demogs = demogs.drop_duplicates() 

# Remove NaNs
demogs.fillna('', inplace=True)

# Create DatFrame for fuxxy matching (pairs) and DataFrame with more than 2 instances 
demogs_pairs = demogs[demogs.groupby("GY_Number")['GY_Number'].transform('size') == 2]
demogs_more_than_2_instances = demogs[demogs.groupby("GY_Number")['GY_Number'].transform('size') > 2]

# Group paird DataFrame by SSN
groups_paris = demogs_pairs.groupby('GY_Number') 


master_list = []
for group_name, df_group in groups_paris:
    # Each group as a list (group) of lists (rows)
    gy_group_list = df_group.astype(str).values.tolist() 
    master_list.append(gy_group_list)

# Define new columns
cols = demogs.columns.tolist()
cols.append('Ratio')
cols.append('Diffs')
cols.append('NoInstances')

# Create new DataFrame to hold results
df_results = pd.DataFrame(columns = cols)

master_score = []

# For each list (group) in master_list
for i in range(len(master_list)):
    score_list = []
    avg_score = []
    diffs_list = []
    
    # For each row in group minus URN
    for x, y in zip(master_list[i][0][1:], master_list[i][1][1:]): 
        score = fuzz.ratio(x, y)
        score_list.append(score)
        
        # Get diffrences between feilds
        splitB = set(x.split(' '))
        splitA = set(y.split(' '))
        diff = splitB.symmetric_difference(splitA)
        if diff != '':
            diff = ", ".join(diff) 
            diffs_list.append(diff)
    
    # Score out of 100
    avg_score = sum(score_list)/len(score_list)
    
    # Append new information to orignal rows
    master_list[i][0].append(int(avg_score))
    master_list[i][1].append(int(avg_score))

    # Remove empty strings
    diffs_list = list(filter(None, diffs_list))

    diffs_string = "        ".join(diffs_list)
    master_list[i][0].append(diffs_string)
    master_list[i][1].append(diffs_string)
    
    master_list[i][0].append('')
    master_list[i][1].append('')

    #Append final lists to DataFrame
    df_results.loc[len(df_results)] = master_list[i][0]
    df_results.loc[len(df_results)] = master_list[i][1]
    

# Append more thank 2 instances DataFrameto results
df_results = df_results.append(demogs_more_than_2_instances.astype(str))

# Count number of instances of GY numbers and add to DataFrame
df_results['NoInstances'] = df_results['GY_Number'].map(df_results['GY_Number'].value_counts())

# Sort results
df_results.sort_values(by= ['Ratio', 'GY_Number'], ascending = [False, False], inplace = True) 

# Re-order Columns
df_results = df_results[['URN',
 'FirstName',
 'LastName',
 'DOB',
 'Address1',
 'Address2',
 'Gender',
 'GY_Number',
 'Ratio',
 'Diffs',
 'NoInstances']]

# To Excel..
writer = pd.ExcelWriter("GY_Number_Dups.xlsx", engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine

# Convert  dataframe to  XlsxWriter Excel object. Turn off the default
# header and index and skip one row to allow us to insert a user defined
# header
df_results.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False, index=False)

# Get the xlsxwriter workbook and worksheet objects
workbook = writer.book 
worksheet = writer.sheets['Sheet1']

# Get  dimensions of df_results
(max_row, max_col) = df_results.shape 

# Create a list of column headers, to use in add_table() 
column_settings = [] 
for header in df_results.columns:
    column_settings.append({'header': header})

# Add table to Excel as per dimensions of results_df
worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings,
                                                'style': 'Table Style Light 11'})

# Make the columns wider for clarity
worksheet.set_column(0, max_col - 1, 12) 

# Close the Pandas Excel writer and output the Excel file
writer.save() 

In [4]:
df_results

,URN,FirstName,LastName,DOB,Address1,Address2,Gender,GY_Number,Ratio,Diffs,NoInstances
594,323603,Henry,Watkin,2006-07-13,Les Cottes,"La Rue Des Cottes, St. Sampson",Male,GY763299,100,,2
595,325132,Henry,Watkin,2006-07-13,Les Cottes,"La Rue Des Cottes, St. Sampson",Male,GY763299,100,,2
588,319398,Ronald,Moniz,1993-12-02,Caledonia Lodge,"Valnord Hill, St. Peter Port",Male,GY762290,100,,2
589,328041,Ronald,Moniz,1993-12-02,Caledonia Lodge,"Valnord Hill, St. Peter Port",Male,GY762290,100,,2
284,88694,Stephanie,Laband,1995-11-02,Quarry Cottage,"Valongis, Alderney",Female,GY677817,100,,2
285,328728,Stephanie,Laband,1995-11-02,Quarry Cottage,"Valongis, Alderney",Female,GY677817,100,,2
218,73293,Ines,Romanillos,1986-06-05,"Flat 2, Rosaire","Corbin Steps, St. Peter Port",Female,GY658629,100,,2
219,322047,Ines,Romanillos,1986-06-05,"Flat 2, Rosaire","Corbin Steps, St. Peter Port",Female,GY658629,100,,2
214,83840,Claire,Robson,1987-10-14,"Seabreeze, Clos Des Sablons","Sandy Lane, St. Sampson",Female,GY658165,100,,2
215,315280,Claire,Robson,1987-10-14,"Seabreeze, Clos Des Sablons","Sandy Lane, St. Sampson",Female,GY658165,100,,2


In [3]:
demogs

,URN,FirstName,LastName,DOB,Address1,Address2,GY_Number,Gender
0,84681,Maxine,Bell,1991-04-17,"Flat 3B, 3 Mansell Street",St. Peter Port,GY669021,Female
1,82257,Melissa,Chamberlain,1969-09-13,La Breloque,Grande Rue Vale,GY617056,Female
2,42047,Michael,Bell,1980-11-04,Dijon,St. Clements Road St. Peter Port,GY651542,Male
3,65389,Peter,Bell,1948-04-24,Beverly Cottage,"Ville Amphrey, St. Martin",GY557960,Male
4,52252,Philip,Bell,1982-06-18,Le Recoin 1 Port Vase Cottages,Port Vase St. Peter Port,GY651543,Male
...,...,...,...,...,...,...,...,...
86340,329947,Jagdeep,Cheema,1977-11-30,Wayside Cheer Hotel,"Les Grandes Rocques, Castel",GY772693,Male
86341,329948,Sophie,Smith,1992-04-17,"Apartment 90, La Charroterie Mills","La Charroterie, St. Peter Port",GY771543,Female
86342,329949,Ketija,Cesniece,2001-09-03,"8, Baugy Estate","Route de Jerbourg, St. Martin",GY766255,Female
86343,329952,Jane,McMeechan,2002-08-28,3 Le Friquet,"Rue Des Bailleuls, St. Andrew",GY771526,Female
